# **Feature Selection and Engineering**
---

In [1]:
import pandas as pd
import ordinal_ranking as rank # Move a copy of ordinal_ranking.py to this directory for this import to work

In [2]:
RANDOM_STATE = 42

In [3]:
flights_test = pd.read_csv('../files/flights_test_no_missing.csv')
feature_space = list(flights_test.columns)
feature_space.append('arr_delay') # Add the target.

flights = pd.read_csv('../files/flights_no_missing.csv')
flights_subspace = flights[feature_space].copy()
flights_subspace.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay
0,2019-05-19,UA,UA_CODESHARE,UA,4264,EV,N48901,4264,12266,IAH,...,12915,LCH,"Lake Charles, LA",1020,1112,N,52.0,1,127,-2.0
1,2019-05-19,UA,UA_CODESHARE,UA,4266,EV,N12540,4266,13244,MEM,...,12266,IAH,"Houston, TX",1148,1340,N,112.0,1,468,-14.0
2,2019-05-19,UA,UA_CODESHARE,UA,4272,EV,N11164,4272,12266,IAH,...,11042,CLE,"Cleveland, OH",1155,1551,N,176.0,1,1091,4.0
3,2019-05-19,UA,UA_CODESHARE,UA,4281,EV,N13995,4281,11042,CLE,...,11278,DCA,"Washington, DC",839,959,N,80.0,1,310,-20.0
4,2019-05-19,UA,UA_CODESHARE,UA,4286,EV,N13903,4286,13061,LRD,...,12266,IAH,"Houston, TX",710,826,N,76.0,1,301,-1.0


Estimate of prediction set size relative to train/test/validation set:

In [4]:
flights_test.shape[0] / flights.shape[0] * 100

28.165286444822605

### Objective:

Our aim for this section is to use the feature space available to the **flights_test** dataframe containing the target range of dates for our prediction submission (January 1 - 7, 2020, inclusive) to select and engineer a range of fetures that best capture the common causes for flight delays. The prediction submission dates are to be treated as if they are in the future for the purpose of generating an appropriate prediction; as such, no historic data for these dates should be considered. Features will be engineered and selected on the basis of how they best captured the different delay classes available to us during the exploration phase of the study.

## **flights**

Features considered here should be engineered and selected only from the subset present in both **flights** and **flights_test**. Since we know the **flights_test** feature space is a subset of the **flights** feature space, we will simply use the columns in flights_test to filter the feature space of flights to prepare our data for transformation and modelling.

### Feature Selection:

* all ids can be discarded initially (except for the `origin` and `dest` ids), but they may also be ranked by delay if that turns out to be interesting/feasible
* `flights` column is always 1; completely uninformative
* dest and origin city names information is already captured by origin and destination, but the `state` could be useful in a later iteration (e.g. binarized to something like `busy_state` for high traffic origins/destinations)

In [5]:
to_drop = [
    'mkt_unique_carrier',
    'mkt_carrier',
    'mkt_carrier_fl_num',
    'op_unique_carrier',
    'tail_num',
    'op_carrier_fl_num',
    'origin',
    'dest',
    'dest_city_name',
    'origin_city_name',
    'flights']

submission_columns = ['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest']

flights_test_drop = set(to_drop) - set(submission_columns)

flights_subspace = flights_subspace.drop(columns=to_drop)
flights_test = flights_test.drop(columns=flights_test_drop)

### Feature Engineering

#### Ranked categorical features:

In practice, we will always know the planned origin and destination as well as the date of any scheduled flight. It is for this reason that we can confidently engineer our ranking columns prior to splitting the data as these effectively constitute new features that we will always have prior knowledge of when a given flight is scheduled.

`fl_date` can be used to add `day_of_week`, and `month_of_year` columns. These could potentially be reordered by rank of expected delay as well as long as each change is noted for the sake of interpretability.

In [6]:
flights_subspace['day_of_week'] = pd.to_datetime(flights_subspace['fl_date']).dt.dayofweek
flights_subspace['month_of_year'] = pd.to_datetime(flights_subspace['fl_date']).dt.month

flights_test['day_of_week'] = pd.to_datetime(flights_test['fl_date']).dt.dayofweek
flights_test['month_of_year'] = pd.to_datetime(flights_test['fl_date']).dt.month

Now let us check the relationship with arrival delay for these features.

In [7]:
flights_subspace.groupby('day_of_week')['arr_delay'].mean().rank()

day_of_week
0    5.0
1    3.0
2    2.0
3    7.0
4    6.0
5    1.0
6    4.0
Name: arr_delay, dtype: float64

In [8]:
flights_subspace.groupby('month_of_year')['arr_delay'].mean()

month_of_year
1      3.923047
2      6.693371
3      2.902349
4      4.071800
5      6.448396
6     10.562826
7      8.871938
8      8.883695
9      1.694430
10     2.810083
11     3.071748
12     5.087656
Name: arr_delay, dtype: float64

As we can see above, although we can separate into discrete ranks by mean, some classes are much closer on average than others which is an inherent drawback of this method.

`origin_airport_id` and `dest_airport_id` could also be ordinalized based on expected delay. IDs should be used and the codes discarded becasue these will always refer to the expected airport across years.

In [9]:
categorical_to_ordinal = ['origin_airport_id',
                          'dest_airport_id',
                          'month_of_year',
                          'day_of_week']

# First the core data.
ordinal_df, ordinal_mappings = rank.rank_features_by_mean(flights_subspace, categorical_to_ordinal, 'arr_delay')
ordinal_df.head()

,origin_airport_id_rank,dest_airport_id_rank,month_of_year_rank,day_of_week_rank
0,205.0,62.0,8.0,4.0
1,194.0,196.0,8.0,4.0
2,205.0,235.0,8.0,4.0
3,138.0,182.0,8.0,4.0
4,290.0,196.0,8.0,4.0


In [10]:
ordinal_df.shape

(2339962, 4)

We then combine the ordinalized features with the training set and drop the untransformed categorical columns.

In [11]:
flights_subspace = pd.concat([
    flights_subspace.drop(columns=categorical_to_ordinal),
    ordinal_df],
    axis=1)
    
# Sanity check
print(flights_subspace.shape)
flights_subspace.head()

(2339962, 12)


,fl_date,branded_code_share,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,distance,arr_delay,origin_airport_id_rank,dest_airport_id_rank,month_of_year_rank,day_of_week_rank
0,2019-05-19,UA_CODESHARE,1020,1112,N,52.0,127,-2.0,205.0,62.0,8.0,4.0
1,2019-05-19,UA_CODESHARE,1148,1340,N,112.0,468,-14.0,194.0,196.0,8.0,4.0
2,2019-05-19,UA_CODESHARE,1155,1551,N,176.0,1091,4.0,205.0,235.0,8.0,4.0
3,2019-05-19,UA_CODESHARE,839,959,N,80.0,310,-20.0,138.0,182.0,8.0,4.0
4,2019-05-19,UA_CODESHARE,710,826,N,76.0,301,-1.0,290.0,196.0,8.0,4.0


In [12]:
# Checking if the origin IDs in test are necessarily a subset of those in flights.
set(flights_test['origin_airport_id'].unique()).issubset(
    set(flights['origin_airport_id'].unique()))

False

A shortcoming of this strategy is that we are not able to properly rank IDs for which we have no prior delay data. in these cases there are a few options: we can assign unknown cases a new rank of 0, or we could assign the mean/median rank to them. Let us first check the proportion of missing values for each column.

In [13]:
for feature, mapping in ordinal_mappings.items():
    flights_test[f'{feature}_rank'] = flights_test[feature].map(mapping)

flights_test.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,distance,day_of_week,month_of_year,origin_airport_id_rank,dest_airport_id_rank,month_of_year_rank,day_of_week_rank
0,2020-01-01 00:00:00,WN,WN,5888,13891,ONT,14771,SFO,1810,1945,N,95,363,2,1,92.0,324.0,5.0,2.0
1,2020-01-01 00:00:00,WN,WN,6276,13891,ONT,14771,SFO,1150,1320,N,90,363,2,1,92.0,324.0,5.0,2.0
2,2020-01-01 00:00:00,WN,WN,4598,13891,ONT,14831,SJC,2020,2130,N,70,333,2,1,92.0,79.0,5.0,2.0
3,2020-01-01 00:00:00,WN,WN,4761,13891,ONT,14831,SJC,1340,1455,N,75,333,2,1,92.0,79.0,5.0,2.0
4,2020-01-01 00:00:00,WN,WN,5162,13891,ONT,14831,SJC,915,1035,N,80,333,2,1,92.0,79.0,5.0,2.0


In [14]:
for feature in ordinal_mappings.keys():
    # Print the missing value percentage for each missing feature.
    print(flights_test[
        flights_test[f'{feature}_rank'].isna()
        ].shape[0] / flights_test.shape[0] * 100)
        
    # Impute the median rank of the feature for each missing value.
    flights_test[f'{feature}_rank'] = flights_test[f'{feature}_rank'].fillna(
        flights_test[f'{feature}_rank'].median())

0.01335241109646055
0.01350414304073851
0.0
0.0


Only a small proportion of data are unknown, this justifies our median rank imputation in both cases.

#### Binary encodings

`dup` and `branded_code_share` can be binarized.

since `dup` is already binay, it just needs to be encoded with 0 and 1.

In [16]:
flights_subspace['dup'] = flights_subspace['dup'].apply(
    lambda binary: 1 if (binary.strip().lower() == 'y') else 0)
    
flights_subspace.head()

,fl_date,branded_code_share,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,distance,arr_delay,origin_airport_id_rank,dest_airport_id_rank,month_of_year_rank,day_of_week_rank
0,2019-05-19,UA_CODESHARE,1020,1112,0,52.0,127,-2.0,205.0,62.0,8.0,4.0
1,2019-05-19,UA_CODESHARE,1148,1340,0,112.0,468,-14.0,194.0,196.0,8.0,4.0
2,2019-05-19,UA_CODESHARE,1155,1551,0,176.0,1091,4.0,205.0,235.0,8.0,4.0
3,2019-05-19,UA_CODESHARE,839,959,0,80.0,310,-20.0,138.0,182.0,8.0,4.0
4,2019-05-19,UA_CODESHARE,710,826,0,76.0,301,-1.0,290.0,196.0,8.0,4.0


In [20]:
flights_subspace['branded_code_share'].unique()

array(['UA_CODESHARE', 'AA_CODESHARE', 'HA', 'WN', 'UA', 'NK', 'AA',
       'DL_CODESHARE', 'B6', 'AS_CODESHARE', 'DL', 'G4', 'AS', 'F9',
       'HA_CODESHARE', 'VX'], dtype=object)

Here we will interpret the presence of _CODESHARE in the class as 1 and the absence as 0.

In [ ]:
flights_subspace['branded_code_share'] = flights_subspace['branded_code_share'].apply(
    lambda codeshare: 1 if (len(codeshare.split('_')) == 2) else 0)
    
flights_subspace.head()

#### Storing the engineered data

In [ ]:
flights_subspace = flights_subspace.drop('fl_date', axis=1)

flights_subspace.to_csv('../files/flights_engi.csv', index=False)
flights_test.to_csv('../files/flights_test_engi.csv', index=False)